---
title: "Pushing Data to Weights and Biases"
author: "Juma Shafara"
date: "10-30-2024"
---

To add your dataset to wandb, you need to have your wandb api key (like `WANDB_API_KEY = wandb_api_key_value)set in a `.env` file. [Read here to know how](). Then you can upload your dataset as below


In [ ]:
#| default_exp wandb_manager

In [ ]:
# | hide
# | export
import pandas as pd
import os
import wandb
from dotenv import load_dotenv

load_dotenv()

def pushDatasetToWandb(file_path: str, job_type: str, description: str = 'no description',
                  artifact_type: str = 'dataset'):
    """
    Uploads a dataset to Weights & Biases as an artifact.
    
    Args:
        file_path (str): Path to the file to upload.
        job_type (str): The type of job (e.g., "training", "evaluation").
        description (str, optional): Description of the artifact. Defaults to 'no description'.
        artifact_type (str, optional): Type of the artifact (e.g., "dataset"). Defaults to 'dataset'.
    
    Raises:
        ValueError: If file_path or job_type are not provided or if the file cannot be read.
    """
    
    # Check for required arguments
    if not file_path:
        raise ValueError("File path cannot be empty. Please provide a valid path to the data file.")
    
    if not job_type:
        raise ValueError("Job type cannot be empty. Please provide a valid job type.")

    # Initialize Weights & Biases
    try:
        wandb.login(key=os.environ["WANDB_API_KEY"])
        wandb.init(project="Risk Assessment", job_type=job_type)
    except wandb.errors.CommError as e:
        raise ConnectionError("Failed to connect to Weights & Biases. Check your API key and internet connection.") from e

    # Load the data and create artifact
    try:
        data = pd.read_csv(file_path)
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {file_path}. Please check the file path.")
    except pd.errors.EmptyDataError:
        raise ValueError(f"File at {file_path} is empty or invalid.")

    artifact = wandb.Artifact(
        name=f"{job_type}_{os.path.basename(file_path)}",
        type=artifact_type,
        description=description,
    )
    artifact.add_file(file_path)

    # Log artifact and metadata
    try:
        wandb.log_artifact(artifact)
        wandb.log({
            'dataset_size': len(data),
            'num_features': len(data.columns),
            'memory_usage_mb': data.memory_usage(deep=True).sum() / (1024 ** 2)
        })
    finally:
        wandb.finish()


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
from rtvpy.wandb_manager import pushDatasetToWandb

In [ ]:
pushDatasetToWandb(file_path='2022_data_selected.csv', 
            job_type='2022_dataset_for_testing',
            artifact_type='dataset',
            description='sample dataset for testing')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jumashafara/.netrc
wandb: Currently logged in as: rtv (analytic). Use `wandb login --relogin` to force relogin


dataset_size,▁
memory_usage_mb,▁
num_features,▁
dataset_size,7418
memory_usage_mb,2.35131
num_features,24
